In [109]:
import numpy as np
import matplotlib.pyplot as plt
from transform_utils import quat_to_rotvec

In [111]:
data = np.load("rollouts/mode1_xarmclose/episode_1.pkl", "rb", allow_pickle=True)

In [112]:
np.shape(data)

(4933, 17)

In [113]:
from scipy.spatial.transform import Rotation as R

def quaternions_to_rotvecs(data: np.ndarray) -> np.ndarray:
    if data.ndim != 2 or data.shape[1] != 17:
        raise ValueError(f"Expected input shape (N,17), got {data.shape}")

    # Extract the two quaternion blocks: each is shape (N, 4) in (w, x, y, z) order.
    q1 = data[:, 3:7]    # columns 3–6
    q2 = data[:, 12:16]  # columns 12–15

    # Build Rotation objects and convert to rotation vectors
    rot1 = R.from_quat(q1).as_rotvec()  # shape (N, 3)
    rot2 = R.from_quat(q2).as_rotvec()  # shape (N, 3)

    out = np.concatenate([
        data[:, 0:3],     # columns 0,1,2
        rot1,             # 3 columns from quaternion1 → rotvec1
        data[:, 7:12],    # columns 7,8,9,10,11
        rot2,             # 3 columns from quaternion2 → rotvec2
        data[:, 16:17],   # column 16 as a (N,1) slice
    ], axis=1)

    # Verify result has shape (N, 15)
    assert out.shape[1] == 15, f"Output has wrong shape {out.shape}"
    return out


In [114]:
mode1_list = []
mode1_lens = []
for i in range(14):
    try:
        data = np.load("rollouts/mode1_xarmclose/episode_{}.pkl".format(i), "rb", allow_pickle=True)
        data = quaternions_to_rotvecs(data)
        pad_num = 5000 - len(data)
        print(pad_num)
        for _ in range(pad_num):
            data = np.vstack([data, data[-1]])
        mode1_list.append(data)
        mode1_lens.append(len(data))
    except FileNotFoundError:
        pass

67
440
998
755
497
226
86
621
393
500


In [115]:
print("Number of mode 1 episodes:", len(mode1_list))
print("Mode 1 lengths:", mode1_lens)
print("Mode 1 longest episode length:", max(mode1_lens))

Number of mode 1 episodes: 10
Mode 1 lengths: [5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000]
Mode 1 longest episode length: 5000


In [116]:
mode2_list = []
mode2_lens = []
for i in range(11):
    try:
        data = np.load("rollouts/mode2_kinovaclose/episode_{}.pkl".format(i), "rb", allow_pickle=True)
        data = quaternions_to_rotvecs(data)
        pad_num = 5000 - len(data)
        print(pad_num)
        for _ in range(pad_num):
            data = np.vstack([data, data[-1]])
        mode2_list.append(data)
        mode2_lens.append(len(data))
    except FileNotFoundError:
        pass

749
595
353
340
719
385
381
626
329
575


In [117]:
print("Number of mode 2 episodes:", len(mode2_list))
print("Mode 2 lengths:", mode2_lens)
print("Mode 2 longest episode length:", max(mode2_lens))

Number of mode 2 episodes: 10
Mode 2 lengths: [5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000]
Mode 2 longest episode length: 5000


In [118]:
mode3_list = []
mode3_lens = []
for i in range(11):
    try:
        data = np.load("rollouts/mode3_over/episode_{}.pkl".format(i), "rb", allow_pickle=True)
        data = quaternions_to_rotvecs(data)
        pad_num = 5000 - len(data)
        print(pad_num)
        for _ in range(pad_num):
            data = np.vstack([data, data[-1]])
        mode3_list.append(data)
        mode3_lens.append(len(data))
    except FileNotFoundError:
        pass

315
288
729
606
708
394
178
671
318
636


In [119]:
print("Number of mode 3 episodes:", len(mode3_list))
print("Mode 3 lengths:", mode3_lens)
print("Mode 3 longest episode length:", max(mode3_lens))

Number of mode 3 episodes: 10
Mode 3 lengths: [5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000]
Mode 3 longest episode length: 5000


In [120]:
np.shape(mode1_list[0])

(5000, 15)

In [121]:
mode1 = np.array(mode1_list)
mode2 = np.array(mode2_list)
mode3 = np.array(mode3_list)

In [122]:
np.shape(mode1)

(10, 5000, 15)

In [123]:
expert_states = np.concatenate([mode1, mode2], axis=0)
print(np.shape(expert_states))

(20, 5000, 15)


In [124]:
np.save("data/expert_actions_rotvec.npy", expert_states)

In [126]:
pot_states1 = []
pot_states2 = []

for traj in mode1:
    diff_mask = np.abs(traj[:, 6] - traj[0, 6]) > 0.001
    changed_indices = np.nonzero(diff_mask)[0]
    desired_idx = None
    for idx in changed_indices:
        if idx > 0:
            desired_idx = idx
            break
    pot_states1.append(traj[desired_idx, :3])

    diff_mask = np.abs(traj[:, 13] - traj[0, 13]) > 0.001
    changed_indices = np.nonzero(diff_mask)[0]
    desired_idx = None
    for idx in changed_indices:
        if idx > 0:
            desired_idx = idx
            break
    pot_states2.append(traj[desired_idx, 7:10])

for traj in mode2:
    diff_mask = np.abs(traj[:, 6] - traj[0, 6]) > 0.001
    changed_indices = np.nonzero(diff_mask)[0]
    desired_idx = None
    for idx in changed_indices:
        if idx > 0:
            desired_idx = idx
            break
    pot_states1.append(traj[desired_idx, :3])

    diff_mask = np.abs(traj[:, 13] - traj[0, 13]) > 0.001
    changed_indices = np.nonzero(diff_mask)[0]
    desired_idx = None
    for idx in changed_indices:
        if idx > 0:
            desired_idx = idx
            break
    pot_states2.append(traj[desired_idx, 7:10])


In [127]:
np.shape(pot_states2)

(20, 3)

In [128]:
pot_states1 = np.array(pot_states1)
pot_states2 = np.array(pot_states2)

In [129]:
np.shape(pot_states1)

(20, 3)

In [130]:
pot_states = np.hstack([pot_states1, pot_states2])
print(np.shape(pot_states))

(20, 6)


In [131]:
print(pot_states[0])
print(pot_states1[0])
print(pot_states2[0])

[ 0.5027082   0.03255719  0.07587283  0.15143855 -0.48670975  0.06442828]
[0.5027082  0.03255719 0.07587283]
[ 0.15143855 -0.48670975  0.06442828]


In [132]:
np.save("data/pot_states1.npy", pot_states1)
np.save("data/pot_states2.npy", pot_states2)
np.save("data/pot_states.npy", pot_states)